<a href="https://colab.research.google.com/github/zmohammed-moes/Macys_XML/blob/main/macy_inventory_to_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# %% [code]
import pandas as pd
import xml.etree.ElementTree as ET

# Reading the spreadsheet
df_uploaded = pd.read_excel("/content/macyXML_template.xlsx")

# Creating the XML structure using the example format and the data from the spreadsheet
root_element = ET.Element("advice_file", {
    "xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
    "xsi:noNamespaceSchemaLocation": "schema.xsd"
})

# Setting common elements
ET.SubElement(root_element, "advice_file_control_number").text = str(df_uploaded["advice_file_control_number"].iloc[0])
ET.SubElement(root_element, "vendorMerchID").text = df_uploaded["vendorMerchID"].iloc[0]
ET.SubElement(root_element, "vendor").text = str(df_uploaded["vendor"].iloc[0])

for _, row in df_uploaded.iterrows():
    product_element = ET.SubElement(root_element, "product")

    ET.SubElement(product_element, "vendor_SKU").text = str(row["vendor_SKU"])
    ET.SubElement(product_element, "merchant_SKU").text = str(row["merchant_SKU"]) if not pd.isna(row["merchant_SKU"]) else ""
    ET.SubElement(product_element, "UPC").text = str(row["UPC"])
    ET.SubElement(product_element, "available").text = row["available"]
    ET.SubElement(product_element, "qtyonhand").text = str(row["qtyonhand"])
    ET.SubElement(product_element, "description").text = row["description"]

    warehouse_breakdown = ET.SubElement(product_element, "warehouseBreakdown")
    warehouse_element = ET.SubElement(warehouse_breakdown, "warehouse", {
        "warehouse-id": row["warehouse_id"]
    })
    ET.SubElement(warehouse_element, "qtyonhand").text = str(row["warehouse_qtyonhand"])

ET.SubElement(root_element, "advice_file_count").text = str(len(df_uploaded))

# Converting the XML structure to a string and saving it
xml_output = ET.tostring(root_element, encoding="UTF-8", method="xml").decode()

with open("/content/macyXML_template.xml", "w", encoding="utf-8") as xml_file:
    xml_file.write(xml_output)